In [22]:
! pip install -U git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

  Cloning git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to /tmp/pip-req-build-59fanr40
  Running command git clone -q git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git /tmp/pip-req-build-59fanr40


In [2]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud #

#All these packages need to be installed from pip
#For ML
import sklearn
# import sklearn.naive_bayes
import sklearn.tree
import sklearn.ensemble
import sklearn.neural_network
import sklearn.decomposition

import numpy as np #arrays
import matplotlib.pyplot as plt #Plots
import matplotlib.colors # For nice colours
import seaborn #Makes plots look nice, also heatmaps
import scipy as sp #for interp

#These are from the standard library
import collections
import os
import os.path
import random
import re
import glob
import pandas as pd
import requests
import json
import math

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning
%matplotlib inline

In [24]:
! pip3 install transformers

In [3]:
# Read the data
tweets_df = pd.read_csv('week45_10media_df.csv')

In [6]:
tweets_df['tokenized_sents'] = tweets_df['Text'].apply(lambda x: [lucem_illud.word_tokenize(s) for s in lucem_illud.sent_tokenize(x)])
tweets_df['normalized_sents'] = tweets_df['tokenized_sents'].apply(lambda x: [lucem_illud.normalizeTokens(s) for s in x])

left = np.repeat([1,0],5005)
tweets_df['left'] = left

KeyboardInterrupt: ignored

In [ ]:
from transformers import pipeline
nlp = pipeline("sentiment-analysis")
tweets_df['sentiment_result'] = tweets_df['Text'].apply(lambda x: nlp(x)[0])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [7]:
tweets_df['sentiment_label'] = tweets_df['sentiment_result'].apply(lambda x: x['label'])

In [8]:
tweets_df['sentiment_score'] = tweets_df['sentiment_result'].apply(lambda x: x['score'])

In [13]:
tweets_df['positive'] = tweets_df['sentiment_label'].apply(lambda x: x == 'POSITIVE')

In [14]:
tweets_df.groupby('left').mean()['positive']

left
0    0.130470
1    0.223177
Name: positive, dtype: float64

So...Both media are very unhappy, and right-skewed media is worse.

In [17]:
tweets_df.groupby('Media').mean()['positive'].sort_values()

Media
NYPostOpinion    0.107892
BreitbartNews    0.119880
FoxNews          0.127872
NRO              0.138861
democracynow     0.140859
amspectator      0.157842
thedailybeast    0.160839
CNN              0.250749
HuffPost         0.258741
jacobin          0.304695
Name: positive, dtype: float64

In [1]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
inputs = tokenizer(list(tweets_df['Text'][::100]), return_tensors="pt",padding=True)
tokens = inputs.tokens()

outputs = model(**inputs).logits
predictions = torch.argmax(outputs, dim=2)

In [28]:
tweets_df['bertweet_tokenized'] = tweets_df['Text'].apply(lambda x: bertweet_tokenize(x))

KeyboardInterrupt: ignored